---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

def blight_model():
    
#     NOTES
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html 
    le = preprocessing.LabelEncoder()
    
#     reading through the forum, it seems that really 3-4 numerical features may be enough.

# feature selection is the KEY Part according to the forum discussion
# lots of people indicate that 3 - 4 features are all that are a needed to train the model 

# leaning towards violation_zip_code
# deriving late_fee into a label of TRUE or FALSE (0 or 1) if there is a late fee potentially
# zip_code of the violator
# fine_amount

# note from forum:
# I selected 4 features and ran a GridSearchCV to find the best parameters. 
# It took me hours to realize that I was displaying the first column of predict_proba instead of the second one which gave me an AUC of 0.2...

#  RandomForest Classifier with default setting. was a recommendation 

# preprocessing data notes
# dropping columns with lots of NULL /NaN are key...apparenlty there may be some that are straight up all NULL, so not using that feature is obvious

# then converting the null or NaN to something else

# ultimately running the model should be pretty straight forward I guess once everything is processed correclty
#      The target variable is compliance
#     loads dataframes


    train_df = pd.read_csv('train.csv', encoding='latin1')
    

        
    test_df = pd.read_csv('test.csv')
    
    addresses_df = pd.read_csv('addresses.csv')
    
    latlons_df = pd.read_csv('latlons.csv')
      
    
#     FEATURE 1 LATE FEE LABEL

# creating, what I think is, a classifer of either true or false based on if there was a late fee or not
# if there was a late fee, that is, if the amount is greater than 0 for the late fee, it gets a 1, otherwise, it is a 0
    train_df['late_fee_class'] = train_df['late_fee'].apply(lambda x: 1 if x > 0 else 0)
    test_df['late_fee_class'] = test_df['late_fee'].apply(lambda x: 1 if x > 0 else 0)
    
#   columns = ['ticket_id', 'agency_name', 'inspector_name', 'violator_name', 'violation_street_number', 'mailing_address_str_number', 
#      'ticket_issued_date', 'hearing_date', 'violation_code', 'disposition', 'fine_amount', 'admin_fee', 
#      'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'grafitti_status', 
#      ' violation_street_name', ' mailing_address_str_name', ' violation_description', ' violation_zip_code', 
#      ' city', ' state', ' zip_code', ' non_us_str_code', ' country' ]

# dropping columns that are not being used in both train and test sets 
    columns_to_drop = ['agency_name', 'inspector_name', 'violator_name', 'violation_street_number', 
                       'mailing_address_str_number','zip_code', 'violation_code', 'ticket_issued_date', 'hearing_date', 'disposition', 'admin_fee', 
                       'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'grafitti_status', 
                       'violation_street_name', 'mailing_address_str_name', 'violation_description', 
                       'city', 'state', 'violation_zip_code', 'non_us_str_code', 'country']
    
# list of columns to be used in both sets as major features
#       columns_to_keep = ['ticket_id', , 'fine_amount', 'late_fee_class']
# 'zip_code'
# additional columns to drop from the train set 
    
    add_columns_to_drop = ['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'compliance_detail', 'collection_status']
    
    train_df = train_df.drop(columns_to_drop, axis=1)
    train_df = train_df.drop(add_columns_to_drop, axis=1)
    #      training set with no null complaince 
    train_df = train_df[train_df['compliance'].notnull()]
#     this is a mess, it removes any zip_code values that are not a number

#     train_df = train_df[train_df['zip_code'].notna()]
#     train_df = train_df[train_df['zip_code'].apply(lambda x: str(x).isnumeric())]
#     train_df['zip_code'] = train_df['zip_code'].astype(str)
    
#     train_df['zip_code']  = le.fit(train_df['zip_code'])
    
    train_df = train_df.set_index('ticket_id')
    
  
    
    test_df = test_df.drop(columns_to_drop, axis=1)
    test_df  = test_df.set_index('ticket_id')
#     test_df['zip_code'] =  test_df['zip_code'].replace(np.nan, "NA", regex=True).astype('category')
    
    y_train = train_df['compliance']  
    
    
    X_train = train_df.drop(['compliance'], axis=1)
#     X_train['zip_code'] =  X_train['zip_code'].replace(np.nan, "NA", regex=True).astype('category')
#     X_train['zip_code'] = X_train['zip_code'].apply(lambda x: 'NA' if x == 'NA' else str(x)[0:5])
    
    X_train['fine_amount'] =  X_train['fine_amount'].replace(np.nan, "NA", regex=True).astype('category')
    
    

    clf = RandomForestClassifier().fit(X_train, y_train)

    probs =  clf.predict_proba(test_df)[:, 1]
    result = pd.Series(probs, index=test_df.index)
    
#     return  train_df['late_fee_class']
# train_df[['late_fee_class', 'late_fee', 'compliance']]

# violation_code, violation_description
#     return  train_df[['violation_code', 'violation_description', 'compliance']]
#     return X_train.groupby(['violation_zip_code']).sum()
    return result
blight_model()

C:\Users\jfthor\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3215: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


ticket_id
284932    0.060704
285362    0.023235
285361    0.069250
285338    0.060704
285346    0.069250
285345    0.060704
285347    0.057410
285342    0.429721
285530    0.023235
284989    0.031125
285344    0.057410
285343    0.023235
285340    0.023235
285341    0.057410
285349    0.069250
285348    0.060704
284991    0.031125
285532    0.031125
285406    0.031125
285001    0.031125
285006    0.023235
285405    0.023235
285337    0.031125
285496    0.057410
285497    0.060704
285378    0.023235
285589    0.031125
285585    0.060704
285501    0.069250
285581    0.023235
            ...   
376367    0.037920
376366    0.036222
376362    0.036222
376363    0.060704
376365    0.037920
376364    0.036222
376228    0.036222
376265    0.036222
376286    0.362084
376320    0.036222
376314    0.036222
376327    0.362084
376385    0.362084
376435    0.497151
376370    0.362084
376434    0.057410
376459    0.069250
376478    0.000000
376473    0.036222
376484    0.018033
376482    0.031125
37